In [0]:
#rdd with count of each word in a sentence
text = ["people are not as beautiful as they look,as they walk or as they talk.they are only as beautiful  as they love,as they care as they share."]
rdd = sc.parallelize(text)
words = rdd.flatMap(lambda line: line.split(" "))
words.collect()
pairs = words.map(lambda s:(s,1))
counts = pairs.reduceByKey(lambda a, b: a + b)
counts.collect()

Out[45]: [('are', 2),
 ('', 1),
 ('beautiful', 2),
 ('talk.they', 1),
 ('as', 6),
 ('look,as', 1),
 ('share.', 1),
 ('care', 1),
 ('walk', 1),
 ('only', 1),
 ('people', 1),
 ('love,as', 1),
 ('not', 1),
 ('they', 6),
 ('or', 1)]

In [0]:
#rdd with distinct elements
text = [3,5,6,6,7,8,4,2]
rdd2 = sc.parallelize(text)
distin = rdd2.distinct(4).collect()
distin

Out[36]: [8, 4, 5, 6, 2, 3, 7]

In [0]:
#rdd with distinct elements with string
text = ["Hello Hello, I am a java java developer"]
rdd3 = sc.parallelize(text)
list = rdd3.flatMap(lambda line:line.split(" "))
list.distinct().collect()

Out[38]: ['a', 'developer', 'Hello', 'am', 'I', 'java', 'Hello,']

In [0]:
#rdd with filter operation
text = [56,34,22,66,-89,21,0,55,22]
rdd4 = sc.parallelize(text)
f = rdd4.filter(lambda x:x>21 or x==0).count()
f

Out[42]: 7

In [0]:
text = [("B",2),("A",1),("C",3)]
#text = [(3,6),(7,9),(2,8),(0,9)]
rdd5 = sc.parallelize(text)
result = rdd5.sortByKey()#default ascending, 0 descending 
result.collect()

Out[54]: [('A', 1), ('B', 2), ('C', 3)]

In [0]:
#aggregateBykey first seq will execute for each then its result will be used in combination func to produce output based on requirement
def seqFunc(acc, teams):
  if(acc[1] > teams[1]):
    return acc
  else: 
    return teams

 
def combFunc(acc1, acc2):
  if(acc1[1] > acc2[1]):
    return acc1 
  else:
    return acc2

premierRDD = sc.parallelize([
 ("Arsenal", "2014–2015", 75), ("Arsenal", "2015–2016", 71), ("Arsenal", "2016–2017", 75), ("Arsenal", "2017–2018", 63),
 ("Chelsea", "2014–2015", 87), ("Chelsea", "2015–2016", 50), ("Chelsea", "2016–2017", 93), ("Chelsea", "2017–2018", 70), 
 ("Liverpool", "2014–2015", 62), ("Liverpool", "2015–2016", 60), ("Liverpool", "2016–2017", 76), ("Liverpool", "2017–2018", 75),
 ("M. City", "2014–2015", 79), ("M. City", "2015–2016", 66), ("M. City", "2016–2017", 78), ("M. City", "2017–2018", 100), 
 ("M. United", "2014–2015", 70), ("M. United", "2015–2016", 66), ("M. United", "2016–2017", 69), ("M. United", "2017–2018", 81) 
 ])
premierMap = premierRDD.map(lambda t: (t[0], (t[1], t[2])))

premierMax = premierMap.aggregateByKey(('', 0), seqFunc, combFunc)
for i in premierMax.collect(): print(i)

('M. United', ('2017–2018', 81))
('Liverpool', ('2016–2017', 76))
('M. City', ('2017–2018', 100))
('Chelsea', ('2016–2017', 93))
('Arsenal', ('2016–2017', 75))

In [0]:
seqFunc = (lambda x, y: (x[0] + y[1], x[1] + 1))
combFunc = (lambda x, y: (x[0] + y[0], x[1] + y[1]))
premierAgg = premierMap.aggregateByKey((0,0), seqFunc, combFunc)
for i in premierAgg.collect(): print(i)

('M. United', (286, 4))
('Liverpool', (273, 4))
('M. City', (323, 4))
('Chelsea', (300, 4))
('Arsenal', (284, 4))

In [0]:
def func(a,b):
  return a+b

premierAgg1 = premierMap.reduceByKey(lambda x,y:func(x,y))
#premierAgg1.collect()
premierAgg1.saveAsTextFile("/FileStore/rdddata787/")

In [0]:
%fs

ls dbfs:/FileStore/rdddata787/

path,name,size
dbfs:/FileStore/rdddata787/_SUCCESS,_SUCCESS,0
dbfs:/FileStore/rdddata787/part-00000,part-00000,92
dbfs:/FileStore/rdddata787/part-00001,part-00001,92
dbfs:/FileStore/rdddata787/part-00002,part-00002,91
dbfs:/FileStore/rdddata787/part-00003,part-00003,0
dbfs:/FileStore/rdddata787/part-00004,part-00004,90
dbfs:/FileStore/rdddata787/part-00005,part-00005,90
dbfs:/FileStore/rdddata787/part-00006,part-00006,0
dbfs:/FileStore/rdddata787/part-00007,part-00007,0


In [0]:
#mapPartitions:- execute each partition in parallel rather than single element 
one_through_9 = range(1,10)
parallel = sc.parallelize(one_through_9,3)
def f(iterator): yield sum(iterator)
parallel.mapPartitions(f).collect()

Out[29]: [6, 15, 24]

In [0]:
#mapPartitionsWithIndex specify indexes for each partition
parallel = sc.parallelize(range(1,10),3)
def show(index, iterator): yield 'index: '+str(index)+" values: "+ str(list(iterator))
parallel.mapPartitionsWithIndex(show).collect()


Out[26]: ['index: 0 values: [1, 2, 3]',
 'index: 1 values: [4, 5, 6]',
 'index: 2 values: [7, 8, 9]']

In [0]:
#reduce action
l = (2,4,2,2,4,3)
rdd = sc.parallelize(l)
rdd.reduce(lambda x,y:x+y)#rdd.sum()

Out[48]: 17

In [0]:
#Sample transformation
x = sc.parallelize([1, 2, 3, 4, 5])
y = x.sample(False, 0.4, 42)
print(x.collect())
print(y.collect())

[1, 2, 3, 4, 5]
[4, 5]

In [0]:
#Repartition the RDD according to the given partitioner and, within each resulting partition, sort records by their keys
rdd = sc.parallelize([(0, 5), (3, 8), (2, 6), (0, 8), (3, 8), (1, 3)])
rdd2 = rdd.repartitionAndSortWithinPartitions(None, lambda x: x == 2, True)
rdd2.glom().collect()

Out[65]: [[(0, 5), (0, 8), (1, 3), (3, 8), (3, 8)], [(2, 6)], [], [], [], [], [], []]

In [0]:
l = [4,3,5,50,5,5]
sc.parallelize(['1']).pipe('cat').collect()

Out[2]: ['1']

In [0]:
rdd = sc.parallelize(l)
print(rdd.take(3))
print(rdd.first())
print(rdd.count())
def f(x):
  print(x*3)
rdd.foreach(f)
rdd.collect()

[4, 3, 5]
4
6
Out[3]: [4, 3, 5, 50, 5, 5]

In [0]:
def f(x): print(x)
a = sc.parallelize([1, 2, 3, 4, 5])
a.foreach(f)

In [0]:
#sc.parallelize([10, 1, 2, 9, 3, 4, 5, 6, 7]).takeOrdered(6)
sc.parallelize([10, 1, 2, 9, 3, 4, 5, 6, 7]).takeOrdered(6, key=lambda x: -x)

Out[19]: [10, 9, 7, 6, 5, 4]

In [0]:
#Return an RDD created by coalescing all elements within each partition into a list.
rdd = sc.parallelize([1, 2, 3, 4], 2)
sorted(rdd.glom().collect())

Out[22]: [[1, 2], [3, 4]]

In [0]:
#Return a new RDD that is reduced into numPartitions partitions.
sc.parallelize([1, 2, 3, 4, 5], 3).glom().collect()
#sc.parallelize([1, 2, 3, 4, 5], 3).coalesce(1).glom().collect()

Out[24]: [[1], [2, 3], [4, 5]]

In [0]:
#Return a new RDD that has exactly numPartitions partitions.
#Can increase or decrease the level of parallelism in this RDD. Internally, this uses a shuffle to redistribute data. If you are decreasing the number of partitions in this RDD, consider using coalesce, which can avoid performing a shuffle.
rdd = sc.parallelize([1,2,3,4,5,6,7], 4)
sorted(rdd.glom().collect())
len(rdd.repartition(2).glom().collect())
len(rdd.repartition(10).glom().collect())

Out[28]: 10

In [0]:
#Return the union of this RDD and another one.
rdd = sc.parallelize([1, 1, 2, 3])
rdd.union(rdd).collect()

Out[29]: [1, 1, 2, 3, 1, 1, 2, 3]

In [0]:
#Return the intersection of this RDD and another one. The output will not contain any duplicate elements, even if the input RDDs did.
rdd1 = sc.parallelize([1, 10, 2, 3, 4, 5])
rdd2 = sc.parallelize([1, 6, 2, 3, 7, 8])
rdd1.intersection(rdd2).collect()

Out[30]: [1, 2, 3]

In [0]:
#Return an RDD containing all pairs of elements with matching keys in self and other.
#Each pair of elements will be returned as a (k, (v1, v2)) tuple, where (k, v1) is in self and (k, v2) is in other.
#Performs a hash join across the cluster.
x = sc.parallelize([("a", 1), ("b", 4)])
y = sc.parallelize([("a", 2), ("a", 3),("b",8)])
sorted(x.join(y).collect())

Out[32]: [('a', (1, 2)), ('a', (1, 3)), ('b', (4, 8))]

In [0]:
#cogroup : For each key k in self or other, return a resulting RDD that contains a tuple with the list of values for that key in self as well as other.
x = sc.parallelize([("a", 1), ("b", 4)])
y = sc.parallelize([("a", 2)])
[(x, tuple(map(list, y))) for x, y in sorted(list(x.cogroup(y).collect()))]

Out[35]: [('a', ([1], [2])), ('b', ([4], []))]

In [0]:
#groupBy : Return an RDD of grouped items.
rdd = sc.parallelize([1, 1, 2, 3, 5, 8])
result = rdd.groupBy(lambda x: x % 2).collect()
sorted([(x, sorted(y)) for (x, y) in result])

Out[38]: [(0, [2, 8]), (1, [1, 1, 3, 5])]

In [0]:
#Group the values for each key in the RDD into a single sequence. Hash-partitions the resulting RDD with numPartitions partitions.
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
#sorted(rdd.groupByKey().mapValues(len).collect())
#sorted(rdd.groupByKey().mapValues(sum).collect())
#sorted(rdd.groupByKey().mapValues(list).collect())
sorted(rdd.groupByKey().mapValues(tuple).collect())

Out[41]: [('a', (1, 1)), ('b', (1,))]

In [0]:
#groupWith : Alias for cogroup but with support for multiple RDDs.
w = sc.parallelize([("a", 5), ("b", 6)])
x = sc.parallelize([("a", 1), ("b", 4)])
y = sc.parallelize([("a", 2)])
z = sc.parallelize([("b", 42)])
[(x, tuple(map(list, y))) for x, y in sorted(list(w.groupWith(x, y, z).collect()))]

Out[42]: [('a', ([5], [1], [2], [])), ('b', ([6], [4], [], [42]))]

In [0]:
#flatMapValues : Pass each value in the key-value pair RDD through a flatMap function without changing the keys; this also retains the original RDD’s partitioning.
x = sc.parallelize([("a", ["x", "y", "z"]), ("b", ["p", "r"])])
def f(x): return x
x.flatMapValues(f).collect()

Out[43]: [('a', 'x'), ('a', 'y'), ('a', 'z'), ('b', 'p'), ('b', 'r')]

In [0]:
#flatMap : Return a new RDD by first applying a function to all elements of this RDD, and then flattening the results.
rdd = sc.parallelize([2, 3, 4])
#sorted(rdd.flatMap(lambda x: range(1, x)).collect())
sorted(rdd.flatMap(lambda x: [(x, x), (x, x)]).collect())

Out[48]: [(2, 2), (2, 2), (3, 3), (3, 3), (4, 4), (4, 4)]

In [0]:
#map : Return a new RDD by applying a function to each element of this RDD.
rdd = sc.parallelize(["b", "a", "c"])
sorted(rdd.map(lambda x: (x, 1)).collect())

Out[52]: [('a', 1), ('b', 1), ('c', 1)]

In [0]:
#lookup : Return the list of values in the RDD for key key. This operation is done efficiently if the RDD has a known partitioner by only searching the partition that the key maps to.
l = range(1000)
rdd = sc.parallelize(zip(l, l), 10)
rdd.lookup(42)  # slow
#[42]
sorted = rdd.sortByKey()
sorted.lookup(42)  # fast
#[42]
sorted.lookup(1024)
#[]
rdd2 = sc.parallelize([(('a', 'b'), 'c')]).groupByKey()
list(rdd2.lookup(('a', 'b'))[0])
#['c']

Out[54]: ['c']

In [0]:
#mapValues : Pass each value in the key-value pair RDD through a map function without changing the keys; this also retains the original RDD’s partitioning.
x = sc.parallelize([("a", ["apple", "banana", "lemon"]), ("b", ["grapes"])])
def f(x): return len(x)
x.mapValues(f).collect()

Out[1]: [('a', 3), ('b', 1)]

In [0]:
from operator import add
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
#(rdd.reduceByKey(add).collect())
(rdd.groupByKey().mapValues(sum).collect())

Out[1]: [('a', 2), ('b', 1)]